# Training a Linear Regression Model to Predict GPU Performance 

Welcome to our programming project! This is where you'll get to apply the knowledge you obtained in class on a real world project. While the demos are done using SKLearn for simplicity, these programming projects aim to expose you to PyTorch, an industrial library for Machine Learning.


## Introduction

The GPU is a piece of computing hardware first developed in the early 1990s for accelerating graphical applications. Due to its highly parallel nature, people eventually found great uses of GPUs in areas other than gaming. Today, the GPU market is a 20 billion dollar market that serves the fields of video games, AI, crypto, scientific computing and engineering sectors.

In this project, we will train a linear regression model on a GPU dataset -- our goal is to build a web application that takes in the **process node, die area, memory size,** and the number of **millions of transistors** to predict the **compute performance** of that GPU.

## Importing the Necessary Libraries

This project aims to train a linear regression model on the **AMD Radeon and Nvidia GPU Specifications** dataset by  JetBrains Datalore. Here are a list of libraries used in this project:

- **PyTorch**: A powerful machine learning Library
- **TorchMetrics**: We only use it for its R2score function
- **Pandas** and **Numpy**: Used for easy data storage and linear algebra
- **SKLearn**: Train test split 
- **Matplotlib** and **Bokeh**: For creating static and dynamic visualizations

In [31]:
!pip install torch torchmetrics
!unzip colab

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from bokeh.plotting import show, figure, output_notebook
import torchmetrics


pd.set_option('display.max_colwidth', None)

Archive:  colab.zip
replace datasets/gpus_v2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Loading the Dataset

The first step is to load and visualize the  dataset. This is crucial in any machine learning development process, as this can give you an idea of what features to include or exclude.

In [46]:
gpu_dataset = pd.read_csv("datasets/gpus_v2.csv")
gpu_dataset = gpu_dataset.sample(frac=1).reset_index(drop=True)
gpu_dataset.head(30)

,Manufacturer,Class,Name,Year,Fab,Transistors (mln),Die size,Memory size,Memory speed,GFLOPS,TDP
0,Nvidia,Workstation,Quadro 5000,2011,40,3100,529,2560,120.000,722.30,152.0
1,AMD Radeon,Datacenter,FirePro RG220,2008,55,242,73,512,51.200,80.00,35.0
2,Nvidia,Datacenter,Tesla M1060,2008,55,1400,470,2048,102.400,622.10,188.0
3,Nvidia,Desktop,GeForce GT 1030,2018,14,1800,74,2048,16.800,1059.00,20.0
4,Nvidia,Datacenter,Tesla V100 SXM2 16 GB,2017,12,21100,815,16384,897.000,15670.00,250.0
5,Nvidia,Desktop,GeForce GTS 275,2009,55,1400,470,1792,127.000,674.00,219.0
6,Nvidia,Datacenter,Tesla P4,2016,16,7200,314,8192,192.300,5704.00,75.0
7,AMD Radeon,Workstation,Radeon Pro Duo,2017,14,11400,464,32768,224.000,7168.00,250.0
8,AMD Radeon,Desktop,Radeon HD 2350,2007,65,180,85,64,3.200,42.00,20.0
9,AMD Radeon,Desktop,Radeon R9 285,2014,28,5000,359,3072,176.000,3290.00,190.0


Here, we are going to choose the columns `Transistors (mln)`, `Die size`, `Memory size` and `Fab` to be our **training features** and `GFLOPS` to be our **labels**.

In [47]:
# Creating np.arrays containing our training features and labels
features = gpu_dataset[["Transistors (mln)", "Die size", "Memory size", "Fab"]].to_numpy()
labels = gpu_dataset[['GFLOPS']].to_numpy()

print(f"features shape: {features.shape}")
print(f"labels shape: {labels.shape}\n")

# Train test split
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2)


print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_test shape: {Y_test.shape}")


features shape: (497, 4)
labels shape: (497, 1)

X_train shape: (397, 4)
Y_train shape: (397, 1)
X_test shape: (100, 4)
Y_test shape: (100, 1)


## Prototyping Our First Model (Multi-variate Linear Regression)

Our first model is going to perform Multi-variate linear regression on the data. In PyTorch, we actually have to define our own class (which is a subclass of torch.nn.Module) as well as its `forward()` function.

In [48]:
from torch.nn import Module, MSELoss, Linear
from torch.optim import SGD

class Linreg(Module):
    def __init__(self, inputs, outputs):
        super().__init__()
        """
        Args:
            inputs (int): the input dimensions
            outputs (int): the output dimensions
        """
        
        self.linear = Linear(inputs, outputs)      
        
    def forward(self, x):
        """
        The forward propagation method. This method is used
        by the torch.nn.Module subclass when training or evaluating
        """
        y_pred = self.linear(x)
        return y_pred
    

In [49]:
input_dims, output_dims = (X_train.shape[1], Y_train.shape[1])

# Epochs is the number of passes over our training set
learning_rate = 0.000000005
epochs = 50000

In [50]:
model = Linreg(input_dims, output_dims)

# Optimizing algorithm and loss (optimizing criterion)
optimizer = SGD(model.parameters(), lr=learning_rate)
loss = MSELoss()

In [51]:
# Must convert np.array to torch.tensor
inputs = torch.from_numpy(X_train).float()
labels = torch.from_numpy(Y_train).float()


for i in range(epochs):
        
    # clears gradient buffer from previous iteration
    model.zero_grad()
    # Feed forward
    pred = model(inputs)
    # Calcluate errors
    errors = loss(pred, labels)
    print(f"loss of iteration {i+1}: {errors}")
    # Backpropagation
    errors.backward()
    # update parameters
    optimizer.step()
    
    

Streaming output truncated to the last 5000 lines.
loss of iteration 45001: 5580529.0
loss of iteration 45002: 5580528.0
loss of iteration 45003: 5580526.0
loss of iteration 45004: 5580524.5
loss of iteration 45005: 5580522.5
loss of iteration 45006: 5580521.0
loss of iteration 45007: 5580521.0
loss of iteration 45008: 5580518.0
loss of iteration 45009: 5580517.5
loss of iteration 45010: 5580516.5
loss of iteration 45011: 5580514.5
loss of iteration 45012: 5580513.5
loss of iteration 45013: 5580512.0
loss of iteration 45014: 5580510.0
loss of iteration 45015: 5580508.5
loss of iteration 45016: 5580507.0
loss of iteration 45017: 5580506.5
loss of iteration 45018: 5580504.5
loss of iteration 45019: 5580503.5
loss of iteration 45020: 5580502.0
loss of iteration 45021: 5580501.0
loss of iteration 45022: 5580499.0
loss of iteration 45023: 5580497.0
loss of iteration 45024: 5580495.5
loss of iteration 45025: 5580495.0
loss of iteration 45026: 5580492.5
loss of iteration 45027: 5580491.0
loss

In [52]:
from torchmetrics import R2Score

# Must convert from np.array to torch.tensor 
test_inputs = torch.from_numpy(X_test).float()
test_labels = torch.from_numpy(Y_test).float()

# Sets model to evaluation mode. Evaluation mode turns off
# certain layers such as dropout or batch-norm, as these
# behave differently while training.
model.eval()

# predict and print r2 score
with torch.no_grad():
    test_pred = model(test_inputs)
    r2 = R2Score()
    
    print(r2(test_pred, test_labels).item())

0.7543301582336426


### Observations

Our first attempt at training a linear regression model provided us with a few observations to note. First is that the loss is huge, even after gradient descent. This is because we are working with huge labels and our output space is very large, so do not be alarmed by the loss. It is better to note the R2 score, which our first model achieved 74%, which is fairly good. Lets fill in the observation table:

| Model | observations | R2 |
| :---- | :----------- | :--- |
| Multi-variate Linear Regerssion | Very large cost, used a very small learning rate | 74% |


## Prototyping Our Second Model (Polynomial Regression)

We are going to train a second model, this time using polynomial regression with cubic features. This in theory should be able to fit a more complicated function to the data.

In [53]:
poly = PolynomialFeatures(degree=3)

# convert to polynomial features
X_train_p = poly.fit_transform(X_train)
X_test_p = poly.fit_transform(X_test)
Y_train_p = Y_train
Y_test_p = Y_test

print(f"X_train_p shape: {X_train_p.shape}")
print(f"Y_train_p shape: {Y_train_p.shape}")
print(f"X_test_p shape: {X_test_p.shape}")
print(f"Y_test_p shape: {Y_test_p.shape}")

X_train_p shape: (397, 35)
Y_train_p shape: (397, 1)
X_test_p shape: (100, 35)
Y_test_p shape: (100, 1)


In [54]:
# batch norm is used to normalize the inputs. Don't worry about 
# Explaining this to students, as it will be covered in a later topic
from torch.nn import BatchNorm1d

class Linreg_p(Module):
    def __init__(self, inputs, outputs):
        super().__init__()
        self.linear = Linear(inputs, outputs) 
        self.bn = BatchNorm1d(inputs)
        
    def forward(self, x):
        x = self.bn(x)
        y_pred = self.linear(x)
        return y_pred

In [55]:
input_dims_p, output_dims_p = (X_train_p.shape[1], Y_train_p.shape[1])
learning_rate_p = 0.000001
epochs_p = 50000

In [56]:
model_p = Linreg_p(input_dims_p, output_dims_p)

# Optimizer and loss
optimizer_p = SGD(model_p.parameters(), lr=learning_rate_p)
loss_p = MSELoss()

In [57]:
# Convert to torch.tensor
inputs_p = torch.from_numpy(X_train_p).float()
labels_p = torch.from_numpy(Y_train_p).float()


for i in range(epochs_p):
        
    # removes gradients from previous iteration
    model_p.zero_grad()
    # Feed Forward
    pred_p = model_p(inputs_p)
    # calculate cost
    errors_p = loss(pred_p, labels_p)
    print(f"loss of iteration {i+1}: {errors_p}")
    # Backpropagation
    errors_p.backward()
    # Update parameters
    optimizer_p.step()
    

Streaming output truncated to the last 5000 lines.
loss of iteration 45001: 1761314.625
loss of iteration 45002: 1761309.5
loss of iteration 45003: 1761304.375
loss of iteration 45004: 1761299.375
loss of iteration 45005: 1761294.125
loss of iteration 45006: 1761288.875
loss of iteration 45007: 1761283.875
loss of iteration 45008: 1761278.75
loss of iteration 45009: 1761273.375
loss of iteration 45010: 1761268.5
loss of iteration 45011: 1761263.5
loss of iteration 45012: 1761257.875
loss of iteration 45013: 1761253.125
loss of iteration 45014: 1761247.625
loss of iteration 45015: 1761242.25
loss of iteration 45016: 1761237.25
loss of iteration 45017: 1761232.25
loss of iteration 45018: 1761227.125
loss of iteration 45019: 1761222.125
loss of iteration 45020: 1761216.75
loss of iteration 45021: 1761211.375
loss of iteration 45022: 1761206.75
loss of iteration 45023: 1761201.625
loss of iteration 45024: 1761196.625
loss of iteration 45025: 1761191.375
loss of iteration 45026: 1761186.5
l

In [58]:
# Convert to torch.tensor
test_inputs_p = torch.from_numpy(X_test_p).float()
test_labels_p = torch.from_numpy(Y_test_p).float()

# Eval mode
model_p.eval()

#predict and print r2
with torch.no_grad():
    test_pred_p = model_p(test_inputs_p)
    r2_p = R2Score()
    print(r2_p(test_pred_p, test_labels_p).item())

0.8952794671058655


This looks even better! I think this model is ready for use. Before we save the model, lets record the observations in the table below:

| Model | observations | R2 |
| :---- | :----------- | :--- |
| Multi-variate Linear Regerssion | Very large cost, used a very small learning rate | 74% |
| Polynomial Regression | Cost is smaller but still large. This is no issue | 93% |

In [61]:
savemodel = torch.jit.trace(model_p, torch.rand((1, X_train_p.shape[1])))
savemodel.save("gpu-predictor.pt")

## References

1. https://www.alliedmarketresearch.com/graphic-processing-unit-market
2. https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module